# Spatial Analysis: California Housing Data with GeoPandas

This notebook demonstrates spatial analysis techniques using the California Housing dataset. We'll explore geographic patterns, create interactive maps, and analyze spatial relationships in housing prices.

## Objectives
- Convert tabular data with coordinates to spatial data (GeoDataFrame)
- Create static maps with basemaps using contextily
- Build interactive maps with folium
- Analyze spatial patterns and clusters
- Perform distance-based spatial analysis
- Create choropleth maps showing geographic variation in house values

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Spatial data
import geopandas as gpd
from shapely.geometry import Point

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as cx
import folium
from folium.plugins import HeatMap, MarkerCluster

# Spatial analysis
import mapclassify
from scipy.spatial import distance
from sklearn.cluster import DBSCAN, KMeans

# Machine Learning (for reference)
from sklearn.datasets import fetch_california_housing

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## 2. Load Data and Create GeoDataFrame

We'll load the California Housing dataset and convert it to a GeoDataFrame using the latitude and longitude coordinates.

In [ ]:
# Load the dataset
housing = fetch_california_housing()

# Create a DataFrame
df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Create Point geometries from Longitude and Latitude
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Create GeoDataFrame
# CRS 4326 is WGS84 (standard lat/lon coordinates)
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

print(f"GeoDataFrame created with {len(gdf)} points")
print(f"CRS: {gdf.crs}")
print(f"\nBounds (min_lon, min_lat, max_lon, max_lat):")
print(gdf.total_bounds)
gdf.head()

## 3. Basic Spatial Visualization

Let's start with simple plots to visualize the geographic distribution of the data.

In [ ]:
# Simple plot of all points
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(ax=ax, alpha=0.5, markersize=1, color='blue')
ax.set_title('California Housing Dataset - All Points', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

In [ ]:
# Plot colored by house value
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(ax=ax, 
         column='MedHouseVal', 
         cmap='RdYlGn', 
         legend=True,
         markersize=5,
         alpha=0.6,
         legend_kwds={'label': 'Median House Value ($100k)', 'orientation': 'horizontal'})
ax.set_title('California Housing Prices by Location', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

## 4. Maps with Basemaps (Contextily)

Add real-world context with basemaps from OpenStreetMap and other providers.

In [ ]:
# For performance, let's work with a sample for some visualizations
# Full dataset has 20k+ points which can be slow to render
sample_size = 5000
gdf_sample = gdf.sample(n=sample_size, random_state=42)

# Convert to Web Mercator projection (EPSG:3857) for use with contextily
gdf_sample_merc = gdf_sample.to_crs(epsg=3857)

print(f"Sample size: {len(gdf_sample_merc)} points")
print(f"New CRS: {gdf_sample_merc.crs}")

In [ ]:
# Create map with OpenStreetMap basemap
fig, ax = plt.subplots(figsize=(14, 10))

gdf_sample_merc.plot(ax=ax, 
                     column='MedHouseVal',
                     cmap='RdYlGn',
                     markersize=10,
                     alpha=0.7,
                     legend=True,
                     legend_kwds={'label': 'Median House Value ($100k)', 
                                 'orientation': 'vertical',
                                 'shrink': 0.6})

# Add basemap
cx.add_basemap(ax, 
               source=cx.providers.OpenStreetMap.Mapnik,
               attribution_size=8)

ax.set_title('California Housing Prices with OpenStreetMap Basemap', fontsize=14, pad=20)
ax.set_xlabel('Longitude (Web Mercator)', fontsize=10)
ax.set_ylabel('Latitude (Web Mercator)', fontsize=10)
ax.set_axis_off()  # Remove axis for cleaner look
plt.tight_layout()
plt.show()

In [ ]:
# Compare different regions: SF Bay Area vs LA Area
# SF Bay Area bounds (approximate)
sf_gdf = gdf[(gdf['Latitude'] >= 37.2) & (gdf['Latitude'] <= 38.0) & 
             (gdf['Longitude'] >= -122.6) & (gdf['Longitude'] <= -121.8)]

# LA Area bounds (approximate)
la_gdf = gdf[(gdf['Latitude'] >= 33.7) & (gdf['Latitude'] <= 34.3) & 
             (gdf['Longitude'] >= -118.7) & (gdf['Longitude'] <= -117.9)]

print(f"SF Bay Area points: {len(sf_gdf)}")
print(f"LA Area points: {len(la_gdf)}")

# Convert to Web Mercator
sf_gdf_merc = sf_gdf.to_crs(epsg=3857)
la_gdf_merc = la_gdf.to_crs(epsg=3857)

# Create side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# SF Bay Area
sf_gdf_merc.plot(ax=ax1, 
                 column='MedHouseVal',
                 cmap='RdYlGn',
                 markersize=15,
                 alpha=0.6,
                 legend=True,
                 legend_kwds={'label': 'Median House Value ($100k)'})
cx.add_basemap(ax1, source=cx.providers.OpenStreetMap.Mapnik)
ax1.set_title(f'SF Bay Area Housing Prices\n(n={len(sf_gdf)} properties)', fontsize=12)
ax1.set_axis_off()

# LA Area
la_gdf_merc.plot(ax=ax2, 
                 column='MedHouseVal',
                 cmap='RdYlGn',
                 markersize=15,
                 alpha=0.6,
                 legend=True,
                 legend_kwds={'label': 'Median House Value ($100k)'})
cx.add_basemap(ax2, source=cx.providers.OpenStreetMap.Mapnik)
ax2.set_title(f'LA Area Housing Prices\n(n={len(la_gdf)} properties)', fontsize=12)
ax2.set_axis_off()

plt.tight_layout()
plt.show()

# Compare statistics
print("\nComparison of SF Bay Area vs LA Area:")
print(f"SF Median House Value: ${sf_gdf['MedHouseVal'].median():.2f} x 100k = ${sf_gdf['MedHouseVal'].median()*100:.0f}k")
print(f"LA Median House Value: ${la_gdf['MedHouseVal'].median():.2f} x 100k = ${la_gdf['MedHouseVal'].median()*100:.0f}k")
print(f"\nSF Mean Median Income: ${sf_gdf['MedInc'].mean():.2f}")
print(f"LA Mean Median Income: ${la_gdf['MedInc'].mean():.2f}")

## 5. Interactive Maps with Folium

Create interactive maps that you can zoom and pan.

In [ ]:
# Create a base map centered on California
center_lat = gdf['Latitude'].mean()
center_lon = gdf['Longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], 
               zoom_start=6,
               tiles='OpenStreetMap')

# Add a sample of points to avoid overwhelming the map
sample_for_map = gdf.sample(n=1000, random_state=42)

# Add colored markers based on house value
for idx, row in sample_for_map.iterrows():
    # Color based on value (high = green, low = red)
    if row['MedHouseVal'] > 3.0:
        color = 'green'
    elif row['MedHouseVal'] > 2.0:
        color = 'orange'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6,
        popup=f"Value: ${row['MedHouseVal']*100:.0f}k<br>Income: ${row['MedInc']:.2f}"
    ).add_to(m)

print("Interactive map created! (Displaying 1,000 sample points)")
m

In [ ]:
# Create heatmap of house values
m_heat = folium.Map(location=[center_lat, center_lon], 
                    zoom_start=6,
                    tiles='CartoDB positron')

# Prepare data for heatmap (latitude, longitude, weight)
heat_data = [[row['Latitude'], row['Longitude'], row['MedHouseVal']] 
             for idx, row in gdf.sample(n=5000, random_state=42).iterrows()]

# Add heatmap
HeatMap(heat_data, 
        min_opacity=0.3,
        max_opacity=0.8,
        radius=15,
        blur=20,
        gradient={0.4: 'blue', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}
       ).add_to(m_heat)

print("Heatmap created! Intensity represents house values.")
m_heat

In [ ]:
# Create map with marker clustering
m_cluster = folium.Map(location=[center_lat, center_lon], 
                       zoom_start=6,
                       tiles='OpenStreetMap')

# Create marker cluster
marker_cluster = MarkerCluster().add_to(m_cluster)

# Add markers to cluster (sample for performance)
for idx, row in gdf.sample(n=2000, random_state=42).iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"""<b>Property Details</b><br>
                  Value: ${row['MedHouseVal']*100:.0f}k<br>
                  Median Income: ${row['MedInc']:.2f}<br>
                  House Age: {row['HouseAge']:.0f} years<br>
                  Avg Rooms: {row['AveRooms']:.1f}""",
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(marker_cluster)

print("Clustered marker map created! Zoom in to see individual properties.")
m_cluster

## 6. Choropleth Maps and Classification

Classify house values into categories and create choropleth-style visualizations.

In [ ]:
# Classify house values using different methods
# Natural Breaks (Jenks)
gdf['value_class_jenks'] = mapclassify.NaturalBreaks(gdf['MedHouseVal'], k=5).yb

# Quantiles
gdf['value_class_quantile'] = mapclassify.Quantiles(gdf['MedHouseVal'], k=5).yb

# Equal Interval
gdf['value_class_equal'] = mapclassify.EqualInterval(gdf['MedHouseVal'], k=5).yb

# Create labels
class_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
gdf['value_category'] = pd.Categorical.from_codes(gdf['value_class_jenks'], class_labels)

print("Classification complete!")
print("\nDistribution by category (Natural Breaks):")
print(gdf['value_category'].value_counts().sort_index())

In [ ]:
# Compare different classification methods
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
fig.suptitle('Comparison of Classification Methods', fontsize=16)

# Natural Breaks
gdf.plot(ax=axes[0], 
         column='value_class_jenks', 
         cmap='RdYlGn', 
         markersize=3,
         alpha=0.6,
         legend=True,
         categorical=True)
axes[0].set_title('Natural Breaks (Jenks)')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')

# Quantiles
gdf.plot(ax=axes[1], 
         column='value_class_quantile', 
         cmap='RdYlGn', 
         markersize=3,
         alpha=0.6,
         legend=True,
         categorical=True)
axes[1].set_title('Quantiles')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')

# Equal Interval
gdf.plot(ax=axes[2], 
         column='value_class_equal', 
         cmap='RdYlGn', 
         markersize=3,
         alpha=0.6,
         legend=True,
         categorical=True)
axes[2].set_title('Equal Interval')
axes[2].set_xlabel('Longitude')
axes[2].set_ylabel('Latitude')

plt.tight_layout()
plt.show()

## 7. Spatial Clustering

Identify geographic clusters of similar housing characteristics using DBSCAN and K-Means.

In [ ]:
# Prepare coordinates for clustering
coords = gdf[['Longitude', 'Latitude']].values

# DBSCAN clustering based on geographic proximity
# eps is in degrees (approximately 0.1 degrees ≈ 11 km)
dbscan = DBSCAN(eps=0.1, min_samples=50)
gdf['cluster_dbscan'] = dbscan.fit_predict(coords)

# Count clusters (excluding noise points labeled as -1)
n_clusters = len(set(gdf['cluster_dbscan'])) - (1 if -1 in gdf['cluster_dbscan'] else 0)
n_noise = list(gdf['cluster_dbscan']).count(-1)

print(f"DBSCAN identified {n_clusters} clusters")
print(f"Number of noise points: {n_noise}")
print(f"\nTop 10 clusters by size:")
print(gdf['cluster_dbscan'].value_counts().head(10))

In [ ]:
# Visualize DBSCAN clusters
fig, ax = plt.subplots(figsize=(14, 10))

# Plot noise points in gray
noise = gdf[gdf['cluster_dbscan'] == -1]
noise.plot(ax=ax, color='lightgray', markersize=1, alpha=0.3, label='Noise')

# Plot clusters
clusters = gdf[gdf['cluster_dbscan'] != -1]
clusters.plot(ax=ax, 
              column='cluster_dbscan', 
              cmap='tab20', 
              markersize=5,
              alpha=0.6,
              legend=False)

ax.set_title(f'DBSCAN Spatial Clustering\n({n_clusters} clusters identified)', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

In [ ]:
# K-Means clustering (geographic regions)
n_regions = 8
kmeans = KMeans(n_clusters=n_regions, random_state=42, n_init=10)
gdf['region_kmeans'] = kmeans.fit_predict(coords)

print(f"K-Means identified {n_regions} regions")
print(f"\nRegion sizes:")
print(gdf['region_kmeans'].value_counts().sort_index())

In [ ]:
# Visualize K-Means regions
fig, ax = plt.subplots(figsize=(14, 10))

gdf.plot(ax=ax, 
         column='region_kmeans', 
         cmap='tab10', 
         markersize=3,
         alpha=0.6,
         legend=True,
         categorical=True,
         legend_kwds={'title': 'Region'})

# Add centroids
centroids = kmeans.cluster_centers_
ax.scatter(centroids[:, 0], centroids[:, 1], 
           c='red', marker='X', s=200, 
           edgecolors='black', linewidths=2,
           label='Region Centers', zorder=5)

ax.set_title(f'K-Means Geographic Regions (k={n_regions})', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Analyze characteristics of each region
region_stats = gdf.groupby('region_kmeans').agg({
    'MedHouseVal': ['mean', 'median', 'std'],
    'MedInc': ['mean'],
    'HouseAge': ['mean'],
    'AveRooms': ['mean'],
    'Population': ['sum'],
    'Latitude': ['mean'],
    'Longitude': ['mean']
}).round(2)

region_stats.columns = ['_'.join(col) for col in region_stats.columns]
region_stats['count'] = gdf.groupby('region_kmeans').size()

print("Regional Statistics:")
print(region_stats.to_string())

In [ ]:
# Compare house values across regions
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Regional Comparison of Housing Characteristics', fontsize=16)

# House values by region
gdf.boxplot(column='MedHouseVal', by='region_kmeans', ax=axes[0, 0])
axes[0, 0].set_title('House Values by Region')
axes[0, 0].set_xlabel('Region')
axes[0, 0].set_ylabel('Median House Value ($100k)')

# Median income by region
gdf.boxplot(column='MedInc', by='region_kmeans', ax=axes[0, 1])
axes[0, 1].set_title('Median Income by Region')
axes[0, 1].set_xlabel('Region')
axes[0, 1].set_ylabel('Median Income')

# House age by region
gdf.boxplot(column='HouseAge', by='region_kmeans', ax=axes[1, 0])
axes[1, 0].set_title('House Age by Region')
axes[1, 0].set_xlabel('Region')
axes[1, 0].set_ylabel('Median House Age (years)')

# Average rooms by region
gdf.boxplot(column='AveRooms', by='region_kmeans', ax=axes[1, 1])
axes[1, 1].set_title('Average Rooms by Region')
axes[1, 1].set_xlabel('Region')
axes[1, 1].set_ylabel('Average Rooms per Household')

plt.tight_layout()
plt.show()

## 8. Distance-Based Analysis

Analyze relationships between properties based on geographic distance.

In [ ]:
# Find properties near a specific location (e.g., San Francisco downtown)
# SF downtown: approximately 37.7749° N, 122.4194° W
sf_downtown = Point(-122.4194, 37.7749)

# Calculate distance from SF downtown (in degrees, approximately)
gdf['dist_from_sf'] = gdf.geometry.distance(sf_downtown)

# Find nearest 100 properties
nearest_sf = gdf.nsmallest(100, 'dist_from_sf')

print(f"Found {len(nearest_sf)} properties near SF downtown")
print(f"\nAverage house value near SF downtown: ${nearest_sf['MedHouseVal'].mean()*100:.0f}k")
print(f"Average house value statewide: ${gdf['MedHouseVal'].mean()*100:.0f}k")
print(f"\nPrice premium near SF downtown: {(nearest_sf['MedHouseVal'].mean() / gdf['MedHouseVal'].mean() - 1) * 100:.1f}%")

In [ ]:
# Visualize distance effect on price
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Map showing distance
gdf_merc = gdf.to_crs(epsg=3857)
gdf_merc.plot(ax=axes[0], 
              column='dist_from_sf', 
              cmap='viridis_r', 
              markersize=3,
              alpha=0.5,
              legend=True,
              legend_kwds={'label': 'Distance from SF Downtown'})

# Mark SF downtown
sf_point = gpd.GeoDataFrame([1], geometry=[sf_downtown], crs='EPSG:4326').to_crs(epsg=3857)
sf_point.plot(ax=axes[0], color='red', markersize=200, marker='*', 
              edgecolor='black', linewidth=2, zorder=5, label='SF Downtown')

axes[0].set_title('Distance from SF Downtown')
axes[0].set_axis_off()
axes[0].legend()

# Scatter plot: distance vs price
axes[1].scatter(gdf['dist_from_sf'], gdf['MedHouseVal'], alpha=0.1, s=5)
axes[1].set_xlabel('Distance from SF Downtown (degrees)')
axes[1].set_ylabel('Median House Value ($100k)')
axes[1].set_title('House Value vs Distance from SF Downtown')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze coastal vs inland properties
# California coast is roughly at longitude -120 and west
gdf['is_coastal'] = gdf['Longitude'] < -120.0

coastal = gdf[gdf['is_coastal']]
inland = gdf[~gdf['is_coastal']]

print(f"Coastal properties: {len(coastal):,}")
print(f"Inland properties: {len(inland):,}")
print(f"\nCoastal avg house value: ${coastal['MedHouseVal'].mean()*100:.0f}k")
print(f"Inland avg house value: ${inland['MedHouseVal'].mean()*100:.0f}k")
print(f"\nCoastal premium: {(coastal['MedHouseVal'].mean() / inland['MedHouseVal'].mean() - 1) * 100:.1f}%")

In [ ]:
# Visualize coastal vs inland
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Map
coastal.plot(ax=axes[0], color='blue', markersize=2, alpha=0.5, label='Coastal')
inland.plot(ax=axes[0], color='orange', markersize=2, alpha=0.5, label='Inland')
axes[0].axvline(x=-120, color='red', linestyle='--', linewidth=2, label='Coastal Boundary')
axes[0].set_title('Coastal vs Inland Properties')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].legend()

# Box plot comparison
data_to_plot = [coastal['MedHouseVal'], inland['MedHouseVal']]
axes[1].boxplot(data_to_plot, labels=['Coastal', 'Inland'])
axes[1].set_ylabel('Median House Value ($100k)')
axes[1].set_title('House Value Distribution: Coastal vs Inland')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

In this tutorial, we:

1. ✅ Converted tabular data with coordinates to a GeoDataFrame
2. ✅ Created static maps with contextily basemaps
3. ✅ Built interactive maps using folium (basic, heatmap, and clustered)
4. ✅ Created choropleth-style visualizations with different classification methods
5. ✅ Performed spatial clustering using DBSCAN and K-Means
6. ✅ Conducted distance-based spatial analysis
7. ✅ Analyzed coastal vs inland property differences

**Key Findings:**
- **Geographic clustering**: Major urban centers (SF Bay Area, LA, San Diego) show high housing values
- **Coastal premium**: Properties near the coast are significantly more expensive than inland properties
- **Distance effect**: Proximity to major cities like San Francisco correlates with higher house values
- **Regional variation**: K-Means clustering revealed 8 distinct geographic regions with different housing characteristics
- **Spatial patterns**: DBSCAN identified natural clusters corresponding to major metropolitan areas

**Spatial Analysis Concepts Covered:**
- **CRS (Coordinate Reference Systems)**: Working with EPSG:4326 (WGS84) and EPSG:3857 (Web Mercator)
- **Point geometry**: Creating and manipulating point geometries
- **Classification methods**: Natural Breaks, Quantiles, Equal Interval
- **Spatial clustering**: DBSCAN for density-based clustering, K-Means for region identification
- **Distance calculations**: Point-to-point distance, distance buffers
- **Interactive mapping**: Folium for web-based interactive maps

**Next Steps:**
- Explore spatial autocorrelation (Moran's I)
- Create convex hulls or buffers around clusters
- Perform spatial joins with other geographic data (e.g., county boundaries)
- Build spatial regression models that account for geographic dependencies
- Analyze temporal patterns if time-series data is available
- Export maps and geodata to various formats (GeoJSON, Shapefile, etc.)